In [ ]:
import json, subprocess
import re, ast, io, os
import shutil
from subprocess import Popen
from os import listdir
from os.path import isdir, join
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import time

fileName = {
    "PST":"true.txt",
    "PSF": "false.txt",
#     "Owl": "input.owl",     # the input KG
    "Owl": "proba_input.owl", # the input assertions with probabilities
    "OwlRule": "proba_rule.owl",  # the input rules with probabilities
    "Orig": "original",
    "ABOX": "triples.txt",
    "TBOX": "rules.txt",
    #"EC": "equClass.txt",
    "Repairs": "meta.txt",
    "Proofs": "proofs.txt",
    "NewRepairs": "newRepair.txt",
    #"ES": "entScores",
    "Solutions": "solutions",   # folder of storing fault-free KGs.
    "Archive": "archive",        # folder of faulty KG that are expensive to further repair
    "PABOX": "prob_triples.txt", # triples with probabilities
    "PTBOX": "prob_rules.txt",   # rules with probabilities
    "Prob": "probability",       # obtained by probability.pl Prolog
    "Suff": "prob_suff.txt",     # sufficient theorems with probabilities
    "Incomp": "prob_incomp.txt"  # incompatible theorems with probabilities
}

repFileStructure = {
    "Heuristics": 1,
    "ProtectList": 2,
    "TheoryGraph": 3,
    "RsBanned": 4,
    "RsApplied": 5,
    "RsLen": 6
}


# run the Prolog
# plFile：Prolog file
# predicate：Functions to be executed in prolog
# argument: parameters
def callProlog(ABCPath, plFile, predicate, argument):
    
    # an example of the running statement：
#      run:- main("c:/users/xuejia/a/data/original/triples.txt",
#       "c:/users/xuejia/a/data/original/rules.txt",
#       "c:/users/xuejia/a/data/true.txt",
#       "c:/users/xuejia/a/data/false.txt",
#       "c:/users/xuejia/a/data/original/proofsnew.txt").

    runPredicate ="run:- " + predicate + "(\"" + "\",\"".join(argument).replace("\\","/") + "\")."
    # print("call the prolog statement:")
    # print(runPredicate)
    
    f = open(plFile, 'r') 
    f2 = open('tempRun.pl', 'w')  # A new prolog file containing the new run statements
    f2.write(f.read() + '\n' + runPredicate)
    f.close()
    f2.close()
    proc =subprocess.run(["swipl", "-l", "tempRun.pl", "-g", "run"],
                              timeout=None, cwd = ABCPath, stdout=subprocess.PIPE)
    
    return  proc.stdout.decode("utf-8")


# transform the input files into the formals accepted by the system
# InputFile: proba_input.owl
# InputRule: proba_rule.owl
# TripleF: triples.txt 
# TripleRel: relevantTriples
# RuleF: rules.txt
# ProbF: prob_triples.txt
# ProbRule: prob_rules.txt
def input2prolog(ABCPath, InputFile, InputRule, TripleF, TripleRel, RuleF, ProbF, ProbRule):
    inpF = open(InputFile, "r")
    inputData = inpF.readlines()
    inpRule = open(InputRule, "r")
    inputRuleData = inpRule.readlines()
    allAboxF = open(TripleF, "w")
    relAboxF= open(TripleRel, "w")
    outputRule = open(RuleF, "w")
    probAboxF = open(ProbF, "w")
    probRuleF = open(ProbRule, "w")

    outData = []
    outDataProb = []
    
    i = 0
    while i < len(inputData):
        tripleData = inputData[i].strip().split(",")
        assertion = "[+["+ tripleData[0] + ",["
        m = 1
        while m < len(tripleData)-1:
            assertion = assertion + tripleData[m]
            if m == len(tripleData)-2:
                assertion = assertion + "]]]."
            else:
                assertion = assertion + "],["
            m += 1
        # assertion = "[+[" + tripleData[1] + ",[" + tripleData[0] + "],[" + tripleData[2] + "]]]."
        outData.append(assertion)
        n = len(tripleData)
        # assertionProb = "[" + tripleData[n-1] + ",[+[" + tripleData[1] + ",[" + tripleData[0] + "],[" + tripleData[2] + "]]]]."
        assertionProb = "[" + tripleData[n-1] + "," + assertion[:-1] + "]."
        outDataProb.append(assertionProb)
        i += 1
    
    allAboxF.write("\n".join(outData))
    probAboxF.write("\n".join(outDataProb))

    outRule = []
    outRuleProb = []

    j = 0
    while j < len(inputRuleData):
        ruleData = inputRuleData[j].strip().split(".", 1)
        rule = ruleData[0]
        outRule.append(rule+".")
        ruleProb = "[" + ruleData[1] + "," + rule + "]."
        outRuleProb.append(ruleProb)
        j += 1
    
    outputRule.write("\n".join(outRule))
    probRuleF.write("\n".join(outRuleProb))
    
    inpF.close()
    inpRule.close()
    allAboxF.close()
    relAboxF.close()
    outputRule.close()
    probAboxF.close()
    probRuleF.close()
    
    inpArg = "\", \"".join([TripleF, TripleRel, RuleF])
    proc =subprocess.run(["swipl", "-l", "util.pl", "-g", "removeQuote([\"" +inpArg+ "\"])"],
           timeout=None, cwd = ABCPath, stdout=subprocess.PIPE)

#     inpArg = [TripleF, TripleRel, RuleF]
#     removeQuoteResult = callProlog(ABCPath, "util.pl", "removeQuote", inpArg)
    
    return


# str2list: convert str to list
# input: string return: list
def str2list(string):
    
    lists = []
    if len(string) == 0:
        return lists
    
    if string[0] == "(":
        symbol_pos = "("
        symbol_neg = ")"
    elif string[0] == "[":
        symbol_pos = "["
        symbol_neg = "]"
        
    i = 0
    j = 0
    count_pos = 0
    count_neg = 0
    count = 0
    
    while i < len(string):            
        if string[i] == symbol_pos:
            count_pos += 1
        if string[i] == symbol_neg:
            count_neg += 1
        if count_pos == count_neg and count_pos != count:
            lists.append(string[j:i+1])
            count = count_pos
            j = i+2
        i += 1
        
    return lists


# read the contents of the file proofs.txt to get the theorems and their proofs
# contents: a line in the file proofs.txt
def axiomGet(contents, writePath):
    
    contentsList = str2list(contents)
    if len(contentsList) == 1:
        contentsList = contentsList[0][1:-1]
        contentsList = str2list(contentsList)        
    # contentsList: a theorem and its SL resolution related information    
    
    theorems_num = len(contentsList) # the number of theorems
    theorem_no = 0
    while theorem_no < theorems_num:
        contentsList[theorem_no] = str2list(contentsList[theorem_no])
        while len(contentsList[theorem_no]) == 1:
            contentsList[theorem_no] = contentsList[theorem_no][0][1:-1]
            contentsList[theorem_no] = str2list(contentsList[theorem_no])
        theorem_no += 1
    # contentsList[n] is a list, including the theorem and its proofs
    
    
    for each_theorem in contentsList:
        # split the proofs of a theorem
        each_theorem[1] = str2list(each_theorem[1])
        while len(each_theorem[1]) == 1:
            each_theorem[1] = each_theorem[1][0][1:-1]
            each_theorem[1] = str2list(each_theorem[1])

    
    # obtain the axiom in proofs
    for each_theorem in contentsList:
        proofs_num = len(each_theorem[1])
        proof_no = 0
        if each_theorem[1][proof_no][0] == "(":
            while proof_no < proofs_num:
                each_theorem[1][proof_no] = str2list(each_theorem[1][proof_no])
                while len(each_theorem[1][proof_no]) == 1:
                    each_theorem[1][proof_no] = each_theorem[1][proof_no][0][1:-1]
                    each_theorem[1][proof_no] = str2list(each_theorem[1][proof_no])
                proof_no += 1
                
    
            
    # write theorems and axioms in their proofs into a file
    writeFile = open(writePath, "w")
    for each_theorem in contentsList:
        if np.array(each_theorem[1]).ndim == 1:
            proof_num = 1
            writeFile.write("Theorem: ")
            writeFile.write(each_theorem[0] + "\n")
            writeFile.write("Axiom: ")
            writeFile.write(each_theorem[1][1] + "\n")
            writeFile.write("\n")
        else:
            proof_num = len(each_theorem[1])
            writeFile.write("Theorem: ")
            writeFile.write(each_theorem[0] + "\n")          
            proof_no = 0
            while proof_no < proof_num:
                writeFile.write("Axiom" + str(proof_no + 1) + ": ")
                writeFile.write(each_theorem[1][proof_no][1] + "\n")
                proof_no += 1
            writeFile.write("\n")
            
    writeFile.close()    
     
    return


# output: prob_suff.txt or prob_incomp.txt, including theorems and axioms with probabilities
# return: the value representing the sufficiency and incompatibility of the theory
# proofContent：suffContent or incompContent，one line from the file proofs.txt
# proofPath：the path of the file containing theorems and axioms in their proofs
# probTripleDir：prob_triples.txt
# probRuleDir：prob_rules.txt
# probDir：probability obtained by running probability.pl, containing values related axioms' contribution for each axiom
# trueLength and falseLength: the number of propositions in True set and False set
def computeProb(proofContent, proofPath, probTripleDir, probRuleDir, probDir, trueLength, falseLength):
    if proofContent == "[]":
        proofFile = open(proofPath, "w")
        proofFile.write("There is no theorems.")
        proofFile.close()
        return 0
    else:
        # step 1: obtain theorems and axioms which are stored in the file of the proofPath
        axiomGet(proofContent, proofPath)

        # step 2: obtain probabilities for each axiom
        # prob_triples.txt
        probTripleFile = open(probTripleDir, "r")
        probTriple = probTripleFile.read()
        probTripleFile.close()
        probTriple = probTriple.split("\n")
        for each in probTriple:
            if each == "":
                probTriple = probTriple.remove("")
        # prob_rules.txt
        probRuleFile = open(probRuleDir, "r")
        probRule = probRuleFile.read()
        probRuleFile.close()
        probRule = probRule.split("\n")
        for each in probRule:
            if each == "":
                probRule = probRule.remove("")

        i = 0
        while i < len(probTriple):
            tripleList = []
            probTriple[i] = probTriple[i][1:-2]
            eachTriple = probTriple[i].split(",", 1)
            tripleList.append(eachTriple[0])
            tripleList.append(eachTriple[1])
            probTriple[i] = tripleList
            i += 1

        j = 0
        if probRule != None:          
            while j < len(probRule):
                ruleList = []
                probRule[j] = probRule[j][1:-2]
                eachRule = probRule[j].split(",", 1)
                ruleList.append(eachRule[0])
                ruleList.append(eachRule[1])
                probRule[j] = ruleList
                j += 1
            
        
        ProbFile = open(probDir, "r")
        probability = ProbFile.read()
        ProbFile.close()
        probabilityList = str2list(probability)
        if len(probabilityList) == 1:
            probabilityList = probabilityList[0][1:-1]
            probabilityList = str2list(probabilityList)
        i = 0
        while i < len(probabilityList):
            eachProbability = []
            splitResult = probabilityList[i].split(")",1)
            eachProbability.append(splitResult[0][2:])
            eachProbability.append(splitResult[1][1:-1])
            probabilityList[i] = eachProbability
            i += 1
            

        ProofFile = open(proofPath, "r")
        ProofContent = ProofFile.read()
        ProofFile.close()
        ProofContent = ProofContent.split("\n")
        

        i = 0
        while i < len(ProofContent):
            if ProofContent[i].startswith("Axiom"):
                each = ProofContent[i].split(":")[1].strip()
                originalProb = 0  # 0: new axiom without probability. 1: input aixom with probability
                currentProb = "0.0"
                ProofContent[i] = each + " =" + currentProb
                
                for eachTriple in probTriple: # the input triples
                    if each == eachTriple[1]:
                        originalProb = 1
                        if float(eachTriple[0]) > float(currentProb):
                            ProofContent[i] = each + " =" + eachTriple[0]
                            currentProb = eachTriple[0]

                if originalProb == 0:
                    if probRule != None:        
                        for eachRule in probRule: # the input rule
                            if each == eachRule[1]:
                                originalProb = 1
                                if float(eachRule[0]) > float(currentProb):
                                    ProofContent[i] = each + " =" + eachRule[0]
                                    currentProb = eachRule[0]
                
                if originalProb == 0: # new axiom without input probability
                    for eachProb in probabilityList:
                        if each == eachProb[1]:
                            valueList = eachProb[0].split(",")
                            if trueLength == 0 :
                                trueValue = 1
                            else :
                                trueValue = float(valueList[0]) / trueLength
                            if falseLength == 0 :
                                falseValue = 0
                            else :
                                falseValue = float(valueList[1]) / falseLength
#                             trueValue = (float(valueList[0]) * float(valueList[2])) / pow(trueLength, 2)
#                             falseValue = (float(valueList[1]) * float(valueList[3])) / pow(falseLength, 2)
                            newProb = round(trueValue * (1 - falseValue), 4)
                            ProofContent[i] = each + " =" + str(newProb)
            i += 1
            
        
        # step 3: compute theorem's probability by multiplying axioms' probabilities
        i = 0
        while i < len(ProofContent):
            if ProofContent[i].startswith("Theorem"):
                prob = 1.0
                j = i + 1
                while len(ProofContent[j]) > 0:
                    prob = prob * float(ProofContent[j].split("=")[1])
                    j += 1
                    if j > len(ProofContent):
                        break
                ProofContent[i] = ProofContent[i] + " =" + str(prob)[0:6]
                i = j + 1
            else:
                i += 1      
        
        writeProb = open(proofPath, "w")
        for eachLine in ProofContent:
            writeProb.write(eachLine+"\n")
        writeProb.close()
        
                
        # step 4: the mean value of theorems' probabilities
        
        probN = []
        probSame = [] # theorems with multiple proofs
        i = 0
        tempProofList = ProofContent.copy()
        while i < len(tempProofList):
            if tempProofList[i].startswith("Theorem"):
                tempTheorem1 = tempProofList[i].split(":")[1]
                j = i + 1
                temp = 0
                while j < len(tempProofList):
                    if tempProofList[j].startswith("Theorem"):
                        tempTheorem2 = tempProofList[j].split(":")[1]
                        if tempTheorem1.split("=")[0].strip() == tempTheorem2.split("=")[0].strip():
                            if temp == 0:
                                sameTheorem = []
                                sameTheorem.append(tempTheorem1.split("=")[0].strip())
                                sameTheorem.append(tempTheorem1.split("=")[1].strip())
                                sameTheorem.append(tempTheorem2.split("=")[1].strip())
                                temp = 1
                            else:
                                sameTheorem.append(tempTheorem2.split("=")[1].strip())
                            del tempProofList[j]
                            probSame.append(sameTheorem)
                    j += 1
                if temp == 0:
                    probN.append(tempTheorem1.split("=")[1].strip())
            i += 1
            
        if len(probSame) != 0:        
            for each in probSame:
                multi = 1.0
                i = len(each) - 1
                while i > 0:
                    multi = multi * (1 - float(each[i]))
                    i -= 1
                overall = 1 - multi
                probN.append(str(overall))
            
        sumProb = 0.0
        for eachNum in probN:
            sumProb = sumProb + float(eachNum)
        sumProb = round((sumProb / len(probN)), 4)

        return sumProb
    
    
# the main entrance for applying ABCT, whose source code is in ABCPath, to the knowledge graph stored in file KGF.
# the preferred structure is PosDir, NegDir
# files are in ABCPath/data.

def abct(ABCPath, KGsDir, Thre):
    threshold = Thre
    # temp=0
    abcStart = time.time()
    # initialise main data folder
    # ABCPath = os.path.abspath(os.getcwd())
    # KGsDir = os.path.join(ABCPath, "data/")
    
    if not os.path.exists(KGsDir):
        os.makedirs(KGsDir)
    else:
        subFolders = [join(KGsDir, f) for f in listdir(KGsDir) if os.path.isdir(join(KGsDir, f))]
        [shutil.rmtree(f, ignore_errors=True) for f in subFolders]    # if exist the folder, remove all its subfolders.

    log = open(os.path.join(KGsDir, "log.txt"), "w")
    log.write("start process.\n")

    # name the input files of true, false sets and *.owl KG.
    PosDir = os.path.join(KGsDir, fileName["PST"])
    NegDir = os.path.join(KGsDir, fileName["PSF"])
    # InpDataDir = os.path.join(KGsDir, fileName["Owl"])
    InpOWLDir = os.path.join(KGsDir, fileName["Owl"])
    # InpRuleDir = os.path.join(KGsDir, fileName["TBOX"])
    InpRuleDir = os.path.join(KGsDir, fileName["OwlRule"])
    repairDir = os.path.join(KGsDir, fileName["Repairs"])

    
     # initialise the serial number of KGs and folder name for storing data
    SerialNum = 0
    # generate the first candidate folder
    KGOrig = os.path.join(KGsDir, fileName["Orig"])
    os.mkdir(KGOrig)

    # copy the input file meta to KGOrig
    if os.path.exists(repairDir):
        shutil.copyfile(repairDir, os.path.join(KGOrig, fileName["Repairs"]))
    else:
        f = open(os.path.join(KGOrig, fileName["Repairs"]), 'w')
        f.write('[].\n'*6)
        f.close()

    #create a file to record applied repair plans by appending
    TripleDir = os.path.join(KGOrig, fileName["ABOX"])
    TripleRel = os.path.join(KGOrig, "relevantTriples")
    RuleDir = os.path.join(KGOrig, fileName["TBOX"])
    #EqDir = os.path.join(KGOrig, fileName["EC"])
    ProofDir = os.path.join(KGOrig, fileName["Proofs"])
    #entDir = os.path.join(KGOrig, fileName["ES"])

    ProbTripleDir = os.path.join(KGOrig, fileName["PABOX"])
    ProbRuleDir = os.path.join(KGOrig, fileName["PTBOX"])
    probDir = os.path.join(KGOrig, fileName["Prob"])
    SuffDir = os.path.join(KGOrig, fileName["Suff"])
    IncompDir = os.path.join(KGOrig, fileName["Incomp"])

    # compute the number of propositions in true set and false set
    trueSet = open(PosDir, "r")
    trueContent = trueSet.read()
    trueSet.close()
    falseSet = open(NegDir, "r")
    falseContent = falseSet.read()
    falseSet.close()
    trueList = str2list(trueContent)
    trueLength = len(trueList)
    falseList = str2list(falseContent)
    falseLength = len(falseList)


    # if there is no owl, copy triples.txt to original/triples.txt
    if not os.path.exists(InpOWLDir):
        shutil.copyfile(os.path.join(KGsDir, fileName["ABOX"]), TripleDir)
    else:
        input2prolog(ABCPath, InpOWLDir, InpRuleDir, TripleDir, TripleRel, RuleDir, ProbTripleDir, ProbRuleDir)
    log.write("--------------- Finish input2prolog---------------")
#     print("finish input convert")

    
    # creat files that do no exist.
    iniFiles = [TripleDir, TripleRel, RuleDir, ProofDir, repairDir, probDir, SuffDir, IncompDir, ProbTripleDir, ProbRuleDir]
    for f in iniFiles:
        open(f, "a+").close()

    # Prepare the input for fault detection
    # inputFaultDet = "\", \"".join([TripleDir, RuleDir, PosDir, NegDir, ProofDir])
    # InsuffNum, IncompNum = fault_det(ABCPath, inputFaultDet)
    # faultNum = str(int(InsuffNum) + int(IncompNum))
    inputFaultDet = [TripleDir, RuleDir, PosDir, NegDir, ProofDir]
    faultDetResult = callProlog(ABCPath, "faultDet.pl", "main", inputFaultDet)
    InsuffNum, IncompNum = faultDetResult.split("\n")[-3:-1]
    InsuffNum = InsuffNum.strip()
    IncompNum = IncompNum.strip()
    faultNum = str(int(InsuffNum) + int(IncompNum))

    proofFile = open(ProofDir, "r")
    proofContent = proofFile.read()
    proofFile.close()
    proofContentList = proofContent.split("\n")
    suffContent = proofContentList[0][0:-1]
    insuffContent = proofContentList[1][0:-1]
    incompContent = proofContentList[2][0:-1]

    # "probability.pl"
    argProb = [TripleDir, RuleDir, ProofDir, probDir]
    probabilityResult = callProlog(ABCPath, "probability.pl", "main", argProb)

    # compute the vaule of the theory
    suffProb = computeProb(suffContent, SuffDir, ProbTripleDir, ProbRuleDir, probDir, trueLength, falseLength)
    incompProb = computeProb(incompContent, IncompDir, ProbTripleDir, ProbRuleDir, probDir, trueLength, falseLength)
    theoryProb = round(suffProb - incompProb, 4)


    # rename the folder of the original KG based on fault numbers
    KG_Nunbers = "_".join([str(theoryProb), str(SerialNum), InsuffNum, IncompNum, faultNum])


    KGinp = os.path.join(KGsDir, KG_Nunbers)
    # os.rename(KGOrig, KGinp)
    # copy everything in KGsDir (the entire folder KGinp) into a new folder named after fileName["Orig"]
    shutil.copytree(KGOrig, KGinp)
    # esOrigDir = os.path.join(KGsDir, fileName["Orig"], KG_Nunbers, fileName["ES"])

    # initise the folders for output
    SolArcDir= os.path.join(KGsDir, fileName["Solutions"])
    if not os.path.exists(SolArcDir):
        os.makedirs(SolArcDir)
    else:
        shutil.rmtree(SolArcDir, ignore_errors=True)    # if exist the folder, remove all its subfolders.

    AbandonDir = os.path.join(KGsDir, fileName["Archive"])
    if not os.path.exists(AbandonDir):
        os.makedirs(AbandonDir)
    else:
        shutil.rmtree(AbandonDir, ignore_errors=True)    # if exist the folder, remove all its subfolders.


    # repair the KGOrig
    sorted_files = [KGinp]
    i = 0
    timelog = open(os.path.join(KGsDir, "timelog.txt"), "w")
    while i < len(sorted_files):
        start_time = time.time()
        # backup SerialNum so that whether new KG is found can be checked
        SerialNumBack = SerialNum
        log.write('\n current SerialNum is  '+ str(SerialNum) +'\n')

        KGCurDir = sorted_files[i]
        # read proofs of this faulty theory
        proofDir = os.path.join(KGCurDir, fileName["Proofs"])

        if not os.path.exists(proofDir):
            warnings.warn("A faulty theory without proof files.")
            continue

        [SuffProofs, InsuffProofs, IncompProof] = open(proofDir).read().split("\n")

        # get the file that records the applied repair plans by appending.
        repairFile = os.path.join(KGCurDir, fileName["Repairs"])
        RuleDir = os.path.join(KGCurDir, fileName["TBOX"])
        TripleDir = os.path.join(KGCurDir, fileName["ABOX"])
        # creat a file to recode newly found repair plans.
        # it contains updated heuristics, ProtectList but only the newly unapplied repairPlans.
        repairNewF = os.path.join(KGCurDir, "repairPlans.txt")
        repairGroupNew  = os.path.join(KGCurDir, "repairPlanGroups.txt")
        open(repairNewF, "a").close()
        open(repairGroupNew, "a").close()
        

        # Step 4 find repaire plans
        # 4.1 generate all possible repair plans
        inpRepairGen = [proofDir, PosDir, NegDir, TripleDir, RuleDir, repairFile, repairNewF, repairGroupNew]
#         argRepairGen = "\", \"".join(inpRepairGen)
#         proc =subprocess.run(["swipl", "-l", "repairGen.pl", "-g", "main(\"" + argRepairGen + "\")"],
#                               timeout=None, cwd = ABCPath, stdout=subprocess.PIPE)
#         repGenInfo = proc.stdout.decode("utf-8")
        repairGenResult = callProlog(ABCPath, "repairGen.pl", "main", inpRepairGen)
        repGenInfo = repairGenResult.strip()

        # if the repair cannot be generated
        if "success" not in repGenInfo:
            warnings.warn("A faulty theory that cannot be repaired has been found.")
            log.write('\n' + KGCurDir+ " cannot be repaired.\n")
            return
        # 4.2 try to compute max sets of commuting repair plans.
        try:
            repairNewF2 = os.path.join(KGCurDir, "repairPlansSets.txt")
            inpRepairGen2 = [repairGroupNew, RuleDir, repairNewF2]
            # argRepairGen2 = "\", \"".join(inpRepairGen2)
            # proc =subprocess.run(["swipl", "-l", "repairGen.pl", "-g", "repCombineFile(\"" + argRepairGen2 + "\")"],
            #                       timeout=None, cwd = ABCPath, stdout=subprocess.PIPE)
            # repGenInfo2 = proc.stdout.decode("utf-8")
            repGenInfo2Result = callProlog(ABCPath, "repairGen.pl", "repCombineFile", inpRepairGen2)
            repGenInfo2 = repGenInfo2Result.strip()
            if "success" in repGenInfo2:
                repNewF = open(repairNewF2, 'r')
                repPlanList = repNewF.readlines()
                repNewF.close()
        # Get a list of repair plans
        except:
            print("\nWarning: no max sets of commuting repair plans are found. Will apply repair plans one by one.\n")
            my_file = open(repairNewF, "r")
            repPlanList = my_file.readlines()       # get the list of rpeair plans.
            my_file.close()

        # Step 5 remove the SerialNum KG with KGs generated by applying each repaire plan
        for repPlanTem in tqdm(repPlanList):
            repPlan = repPlanTem.split(".")[0]
            log.write('\n step 5: '+ repPlan + '\n')
            # creat a sub-forder to store files about applying the ith repair plan to the current KG
            KGNewTem = os.path.join(KGCurDir, "repair")
            os.makedirs(KGNewTem)
            repTripleDir = os.path.join(KGNewTem, fileName["ABOX"])
            repRuleDir = os.path.join(KGNewTem, fileName["TBOX"])
            repairFileNew = os.path.join(KGNewTem, fileName["Repairs"])
            # creat files that do no exist.
            newFiles = [repTripleDir, repRuleDir, repairFileNew]
            for f in newFiles:
                open(f, "a+").close()

            inputAppRep = [repPlan, TripleDir, RuleDir, repairFile, repTripleDir, repRuleDir, repairFileNew]
            # ArgAppRep = "\", \"".join(inputAppRep)

            # # Apply repairs to generate a newly repaired KG for the next layer.
            # proc =subprocess.run(["swipl", "-l", "repairApply.pl", "-g", "main(\"" + ArgAppRep + "\")"],
            #                       timeout=None, cwd = ABCPath, stdout=subprocess.PIPE)
            # appResult = proc.stdout.decode("utf-8")
            repairApplyResult = callProlog(ABCPath, "repairApply.pl", "main", inputAppRep)
            appResult = repairApplyResult.strip()

            # if the repair is applied, then generate new folde for the new KG
            if "success" in appResult:
                log.write("\n success in applying " + repPlan +" to " + KGCurDir + "\n")
                timelog = open(os.path.join(KGsDir, "timelog.txt"), "a+")
                timelog.write(str(time.time() - start_time)+"\n")
                timelog.close()
                SerialNum += 1    # confirm that serial number of KG is increased by 1

                # get information about the repairs
                oldRepF = open(repairFile, "r")
                newRepF = open(repairFileNew, "r")
                first = oldRepF.readlines()[:-3]     # depends on repFileStructur
                second= newRepF.readlines()
                newRepF.close()
                oldRepF.close()

                content = first + second

                repFwrite = open(repairFileNew, "w")
                repFwrite.write(''.join(content))
                repFwrite.close()

                # TODO: calculate equivalence classes
                # detect faults
                proofNewDir = os.path.join(KGNewTem, fileName["Proofs"])
                open(proofNewDir, 'w').close()
#                 entDirNew = os.path.join(KGNewTem, fileName["ES"])
#                 open(entDirNew, 'w').close()

                # Prepare the input for fault detection
                # inpFaultDet = "\", \"".join([repTripleDir, repRuleDir, PosDir, NegDir, proofNewDir])
                # InsuffNum, IncompNum  = fault_det(ABCPath, inpFaultDet)      # detect remaining faults, whose proofs are written in the file "faultProofs".
                inpFaultDet = [repTripleDir, repRuleDir, PosDir, NegDir, proofNewDir]
                faultDetResult = callProlog(ABCPath, "faultDet.pl", "main", inpFaultDet)
                InsuffNum, IncompNum = faultDetResult.split("\n")[-3:-1]
                InsuffNum = InsuffNum.strip()
                IncompNum = IncompNum.strip()
                faultNumNew = int(InsuffNum) + int(IncompNum)
                costNew = faultNumNew + int(second[-1][0])  # remaining fault number + applied repairs" number
                log.write('\n faultNumNew is '+ str(faultNumNew) + '\n')

                suffNewDir = os.path.join(KGNewTem, fileName["Suff"])
                incompNewDir = os.path.join(KGNewTem, fileName["Incomp"])
                probDirNew = os.path.join(KGNewTem, fileName["Prob"])
                open(probDirNew, 'w').close()

                proofNewFile = open(proofNewDir, "r")
                proofNewContent = proofNewFile.read()
                proofNewFile.close()
                proofNewContentList = proofNewContent.split("\n")
                suffNew = proofNewContentList[0][0:-1]
                insuffNew = proofNewContentList[1][0:-1]
                incompNew = proofNewContentList[2][0:-1]

                argProbNew = [repTripleDir, repRuleDir, proofNewDir, probDirNew]
                probabilityNew = callProlog(ABCPath, "probability.pl", "main", argProbNew)
                suffProbNew = computeProb(suffNew, suffNewDir, ProbTripleDir, ProbRuleDir, probDirNew, trueLength, falseLength)
                incompProbNew = computeProb(incompNew, incompNewDir, ProbTripleDir, ProbRuleDir, probDirNew, trueLength, falseLength)
                theoryProbNew = round(suffProbNew - incompProbNew, 4)

                KGDirNew = os.path.join(KGsDir, "_".join([str(theoryProbNew), str(SerialNum), InsuffNum, IncompNum, str(faultNumNew)]))
                
                
                os.rename(KGNewTem, KGDirNew)


                # archive the fault-free KGs to the solutions folder.
                if theoryProbNew < threshold:
                    shutil.move(KGDirNew, AbandonDir)
                else:
                    if faultNumNew == 0:
                        log.write('\n find a solution, named '+ KGDirNew + '\n')
                        shutil.move(KGDirNew, SolArcDir)


            # otherwise, log.write error message and then continue to apply the next repair plan
            else:
                shutil.rmtree(KGNewTem)
                print("Error: repair plan" + repPlan +" cannot be applied to " + KGCurDir )
                log.write("\n Error: repair plan: " + repPlan +" cannot be applied to " + KGCurDir + "\n")
                continue


        # archive the faulty KG whose repiars have been explored and the onese whose score is smaller than the threshold
        shutil.move(KGCurDir, AbandonDir)


        # if there is at least one repaired KG" found for the input faulty KG, rerank the current faulty theory and start over
        if SerialNumBack < SerialNum:
            theoryList = [f for f in listdir(KGsDir) if (os.path.isdir(join(KGsDir, f)) and len(f.split('_')) > 3)]           
            num = len(theoryList)
            for i in range(num-1):
                k = i
                for j in range(i+1, num):
                    if float(theoryList[j].split('_')[0]) >= 0:
                        if float(theoryList[j].split('_')[0]) > float(theoryList[k].split('_')[0]):
                            k = j
                    elif float(theoryList[j].split('_')[0]) < 0:
                        if float(theoryList[k].split('_')[0]) < 0:
                            if abs(float(theoryList[j].split('_')[0])) < abs(float(theoryList[k].split('_')[0])):
                                k = j
                if i != k:
                    theoryList[i], theoryList[k] = theoryList[k], theoryList[i]                       
            sorted_files = [join(KGsDir, f) for f in theoryList]
            # print("sorted_files,", sorted_files)
            # if there is no faulty KG, terminate.
            if sorted_files == []:
                break
            log.write("\n current faulty KGs to repiar include: ")
            log.write('\n'.join(sorted_files) + '\n')
            i = 0
        else:
            i += 1

    
    log.write("--------------- Finish Reapir Generation---------------")
    print("--------------- Finish Reapir Generation---------------")

    runningTime = time.time() - abcStart
    log.write("\n Running time is "+ str(runningTime) +" seconds.")
    log.close()
    print("Running time is %s  seconds." %runningTime)

    return runningTime


# if __name__ == '__main__':
#     abct()


In [ ]:
ABCPath = os.path.abspath(os.getcwd())
dataDir = os.path.join(ABCPath, 'data')
Thre = 0.5 # set the threshold
time = abct(ABCPath, dataDir, Thre)